In [1]:
import numpy as np
import math
import time
import random
import pandas as pd
import os

In [2]:
list_of_problems = []
a = os.listdir("Taillard Benchmark") # returns list
a.pop(0)
for i in a:
    list_of_problems.append("Taillard Benchmark/" + i)
list_of_problems

['Taillard Benchmark/ta01.txt',
 'Taillard Benchmark/ta02.txt',
 'Taillard Benchmark/ta03.txt',
 'Taillard Benchmark/ta04.txt',
 'Taillard Benchmark/ta05.txt',
 'Taillard Benchmark/ta06.txt',
 'Taillard Benchmark/ta07.txt',
 'Taillard Benchmark/ta08.txt',
 'Taillard Benchmark/ta09.txt',
 'Taillard Benchmark/ta10.txt',
 'Taillard Benchmark/ta11.txt',
 'Taillard Benchmark/ta12.txt',
 'Taillard Benchmark/ta13.txt',
 'Taillard Benchmark/ta14.txt',
 'Taillard Benchmark/ta15.txt',
 'Taillard Benchmark/ta16.txt',
 'Taillard Benchmark/ta17.txt',
 'Taillard Benchmark/ta18.txt',
 'Taillard Benchmark/ta19.txt',
 'Taillard Benchmark/ta20.txt',
 'Taillard Benchmark/ta21.txt',
 'Taillard Benchmark/ta22.txt',
 'Taillard Benchmark/ta23.txt',
 'Taillard Benchmark/ta24.txt',
 'Taillard Benchmark/ta25.txt',
 'Taillard Benchmark/ta26.txt',
 'Taillard Benchmark/ta27.txt',
 'Taillard Benchmark/ta28.txt',
 'Taillard Benchmark/ta29.txt',
 'Taillard Benchmark/ta30.txt',
 'Taillard Benchmark/ta31.txt',
 'Tailla

In [3]:
list_of_loop_count = [50,100,250,500,1000,2500]
list_of_trial_count = [5,10,15,20,25]

In [4]:
def read_from_file(filename):
    with open(filename, "r") as fic:
        my_number_of_machine = int(fic.readline())
        my_number_of_jobs = int(fic.readline())
        my_optimal_obj = int(fic.readline())
        my_p_ij = np.zeros((my_number_of_machine, my_number_of_jobs))
        for i in range(my_number_of_machine):
            p_i = fic.readline().split()
            for j in range(my_number_of_jobs):
                my_p_ij[i][j] = int(p_i[j])
    return my_number_of_machine, my_number_of_jobs, my_optimal_obj, my_p_ij

In [5]:
# Objective Function
def makespan(my_seq, p_ij, number_of_machine):
    c_ij = np.zeros((number_of_machine, len(my_seq) + 1))

    for j in range(1, len(my_seq) + 1):
        c_ij[0][j] = c_ij[0][j - 1] + p_ij[0][my_seq[j - 1]]

    for i in range(1, number_of_machine):
        for j in range(1, len(my_seq) + 1):
            c_ij[i][j] = max(c_ij[i - 1][j], c_ij[i][j - 1]) + p_ij[i][my_seq[j - 1]]
    # print(c_ij)
    return c_ij

In [6]:
# Fitness Function
def fit(f, optimal_obj = 1000):
    return (optimal_obj/f)

In [7]:
# Crossover Function
def CX(current_sol, partner_sol, rv):
    # Mengambil Urutan Jadwal
    sol = current_sol.copy()
    sol_partner = partner_sol.copy()
    sol = sol.tolist()
    sol_partner = sol_partner.tolist()
    del sol[-3:]
    del sol_partner[-3:]
    
    cycle = []
    cycle.append(current_sol[rv])
    
    temp = partner_sol[rv]
    index = 0
    if(type(temp) == pd.core.series.Series):
        temp = temp.tolist()[0]
    while(temp != cycle[0]):
        cycle.append(temp)
        index = sol.index(temp)
        temp = sol_partner[index]
        if(type(temp) == pd.core.series.Series):
            temp = temp.tolist()[0]
    
    child = sol
    for i in range(len(child)):
        if(child[i] in cycle):
            index = cycle.index(child[i])
            if(len(cycle) == index + 1):
                index = 0
            else:
                index = index + 1
            child[i] = cycle[index]
    
    child = [int(i) for i in child]
    
    obj = makespan(child, p_ij, number_of_machine)[number_of_machine - 1][number_of_jobs]
    child.append(int(obj))
    child.append(fit(obj, optimal_obj))
    child.append(int(0))
    
    dictionary = dict(zip(column_name, child))
        
    df_dictionary = pd.DataFrame([dictionary])
        
    return df_dictionary

In [8]:
def optimality_gap(makespan, optimal_obj):
    if(makespan < optimal_obj):
        return "{:.1%}".format(0)
    else:
        return "{:.1%}".format((makespan - optimal_obj)/optimal_obj)

In [10]:
# Fase Inisialisasi
number_of_machine, number_of_jobs, optimal_obj, p_ij = read_from_file("Taillard Benchmark/ta01.txt")
Npop = 30

In [11]:
# Untuk Fase Inisialisasi
column_name = []
for i in range(number_of_jobs):
    column_name.append("x_" + str(i+1))
x = column_name.copy()
column_name.append("Makespan")
column_name.append("Fitness")
column_name.append("Trial")

def initialization(Npop, column_name):
    population = pd.DataFrame(columns = column_name)
    pop = []
    for i in range(Npop):
        p = list(np.random.permutation(number_of_jobs))
        while p in pop:
            p = list(np.random.permutation(number_of_jobs))
            
        pop.append(p)
        
        obj = makespan(p, p_ij, number_of_machine)[number_of_machine - 1][number_of_jobs]
        p.append(obj)
        p.append(fit(obj, optimal_obj))
        p.append(0)
        dictionary = dict(zip(column_name, p))
        
        df_dictionary = pd.DataFrame([dictionary])
        
        population = pd.concat([population, df_dictionary], ignore_index = True)
    
    return population

In [12]:
solutions_list = ["Problem", "Number of Iterations", "Termination Criterion", "Makespan", "Time (s)", "%Gap"]
problems_solutions = pd.DataFrame(
    columns = solutions_list
)

In [14]:
for problem in list_of_problems:
    number_of_machine, number_of_jobs, optimal_obj, p_ij = read_from_file(problem)
    Npop = 30
    
    column_name = []
    for i in range(number_of_jobs):
        column_name.append("x_" + str(i+1))
    x = column_name.copy()
    column_name.append("Makespan")
    column_name.append("Fitness")
    column_name.append("Trial")
    
    for loop_count in list_of_loop_count:
        for trial_count in list_of_trial_count:
            population = initialization(Npop, column_name)
            newPop = population.copy()
            solutions = pd.DataFrame(columns = column_name)
            t1 = time.process_time()
            for j in range(loop_count):
                # Fase Employed Bee
                for i in range(len(newPop)):
                    rp = i
                    while(rp == i):
                        rp = random.randint(1, Npop) - 1

                    rv = random.randint(1, number_of_jobs) - 1

                    sol = newPop.iloc[i]
                    sol_partner = newPop.iloc[rp]

                    new_sol = CX(sol, sol_partner, rv)

                    if(new_sol.Fitness[0] <= sol.Fitness):
                        sol.Trial = sol.Trial + 1
                    else:
                        sol = new_sol

                    newPop.iloc[i] = sol
                # Fase Onlooker Bee 
                prob = []
                sum_of_fitness = newPop.Fitness.sum()
                for i in range(len(newPop)):
                    prob.append(newPop.Fitness[i]/sum_of_fitness)

                counter = 0
                for i in range(len(newPop)):
                    r = random.random()
                    while(r > prob[counter]):
                        counter = counter + 1
                        if(counter == len(newPop)):
                            counter = 0
                        r = random.random()

                    rp = counter
                    while(rp == counter):
                        rp = random.randint(1, Npop) - 1

                    rv = random.randint(1, number_of_jobs) - 1

                    sol = newPop.iloc[counter]
                    sol_partner = newPop.iloc[rp]

                    new_sol = CX(sol, sol_partner, rv)

                    if(new_sol.Fitness[0] <= sol.Fitness):
                        sol.Trial = sol.Trial + 1
                    else:
                        sol = new_sol

                    newPop.iloc[counter] = sol

                # Fase Scout Bee
                for i in range(len(newPop)):
                    # Cari yang sudah melewati batas Trial
                    if(newPop.Trial[i] > trial_count):
                        # Simpan solusi tersebut
                        solutions = newPop.iloc[i]
                        # Buat calon solusi yang baru
                        temp = initialization(1, column_name)
                        newPop.iloc[i] = temp

            temp = newPop[newPop.Makespan == newPop.Makespan.min()]
            temp = temp.iloc[0]
            if(solutions.empty):
                solutions = temp
            else:
                if(temp.Makespan < solutions.Makespan):
                    solutions = temp
            t2 = time.process_time()
            time_passed = t2 - t1
            gap = optimality_gap(solutions.Makespan, optimal_obj)
            
            data = [problem, loop_count, trial_count, solutions.Makespan, time_passed, gap]
            dictionary = dict(zip(solutions_list, data))
            df_dict = pd.DataFrame([dictionary])
            print(df_dict)
            problems_solutions = pd.concat([problems_solutions, df_dict], ignore_index = True)

problems_solutions

                       Problem  Number of Iterations  Termination Criterion  \
0  Taillard Benchmark/ta01.txt                    50                      5   

   Makespan  Time (s)  %Gap  
0    1297.0  3.546875  1.5%  
                       Problem  Number of Iterations  Termination Criterion  \
0  Taillard Benchmark/ta01.txt                    50                     10   

   Makespan  Time (s)  %Gap  
0    1297.0  3.859375  1.5%  
                       Problem  Number of Iterations  Termination Criterion  \
0  Taillard Benchmark/ta01.txt                    50                     15   

   Makespan  Time (s)  %Gap  
0    1349.0     3.375  5.6%  
                       Problem  Number of Iterations  Termination Criterion  \
0  Taillard Benchmark/ta01.txt                    50                     20   

   Makespan  Time (s)  %Gap  
0    1323.0  3.203125  3.5%  
                       Problem  Number of Iterations  Termination Criterion  \
0  Taillard Benchmark/ta01.txt               

,Problem,Number of Iterations,Termination Criterion,Makespan,Time (s),%Gap
0,Taillard Benchmark/ta01.txt,50,5,1297.0,3.546875,1.5%
1,Taillard Benchmark/ta01.txt,50,10,1297.0,3.859375,1.5%
2,Taillard Benchmark/ta01.txt,50,15,1349.0,3.375,5.6%
3,Taillard Benchmark/ta01.txt,50,20,1323.0,3.203125,3.5%
4,Taillard Benchmark/ta01.txt,50,25,1318.0,3.046875,3.1%
...,...,...,...,...,...,...
115,Taillard Benchmark/ta81.txt,2500,5,6882.0,758.828125,9.5%
116,Taillard Benchmark/ta81.txt,2500,10,6854.0,707.4375,9.0%
117,Taillard Benchmark/ta81.txt,2500,15,6760.0,648.125,7.5%
118,Taillard Benchmark/ta81.txt,2500,20,6754.0,689.09375,7.4%


In [22]:
problems_solutions[problems_solutions.Problem == list_of_problems[0]]

,Problem,Number of Iterations,Termination Criterion,Makespan,Time (s),%Gap
0,Taillard Benchmark/ta01.txt,10,5,1349.0,0.71875,5.6%
1,Taillard Benchmark/ta01.txt,10,10,1315.0,0.703125,2.9%
2,Taillard Benchmark/ta01.txt,10,20,1339.0,0.625,4.8%
3,Taillard Benchmark/ta01.txt,50,5,1297.0,3.796875,1.5%
4,Taillard Benchmark/ta01.txt,50,10,1339.0,3.765625,4.8%
5,Taillard Benchmark/ta01.txt,50,20,1304.0,3.265625,2.0%
6,Taillard Benchmark/ta01.txt,100,5,1297.0,7.3125,1.5%
7,Taillard Benchmark/ta01.txt,100,10,1323.0,7.0625,3.5%
8,Taillard Benchmark/ta01.txt,100,20,1297.0,6.546875,1.5%
9,Taillard Benchmark/ta01.txt,200,5,1338.0,15.03125,4.7%


In [15]:
problems_solutions.to_csv("Experiment Results.csv", index = False)